In [68]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

Ruta de los datasets

In [69]:
ruta_steam_games = 'datasets/output_steam_games.json'
ruta_users_items = 'datasets/australian_users_items.json'
ruta_users_reviews = 'datasets/australian_user_reviews.json'

ruta_parquet_games = "datasets/data_steam_games.parquet"
ruta_parquet_reviews = "datasets/data_reviews.parquet"
ruta_parquet_items = "datasets/data_items.parquet"

ruta_games_csv = 'datasets/steam_games.csv'
ruta_items_csv = 'datasets/users_items.csv'
ruta_reviews_csv = 'users_reviews.csv'

Importación de Dataset

In [81]:
df_games = pd.read_parquet(ruta_parquet_games)

Cálculo de la Similitud del Coseno

In [71]:

# Crear una copia del DataFrame para evitar modificar el original
df_steam_copy = df_games.copy()

# Desanidar las columnas de género
df_steam_copy['genres'] = df_steam_copy['genres'].apply(eval)  # Convierte las listas de géneros en listas de Python
unique_genres = list(set(genre for sublist in df_steam_copy['genres'] for genre in sublist))

# Crear columnas binarias para cada género
mlb = MultiLabelBinarizer()
genre_features = pd.DataFrame(mlb.fit_transform(df_steam_copy['genres']), columns=mlb.classes_)

Función de recomendación

In [76]:
def recomendacion_juego(id_producto, num_recomendaciones=5):
    # Verificar si el ID del juego está en el índice de genre_features
    if id_producto not in genre_features.index:
        return {"error": "El juego no se encuentra en la base de datos."}

    # Resto del código para calcular la similitud del coseno y obtener las recomendaciones
    juego_seleccionado = np.array(genre_features.loc[id_producto].values).reshape(1, -1)
    similaridades = cosine_similarity(juego_seleccionado, genre_features)

    juegos_similares_indices = similaridades.argsort()[0][-num_recomendaciones:][::-1]
    juegos_recomendados = df_steam_copy.iloc[juegos_similares_indices, :]

    lista = [{"item_id": row['item_id'], "title": row['title']} for index, row in juegos_recomendados.iterrows()]

    return lista


In [78]:
recomendacion = recomendacion_juego(50)
recomendacion

[{'item_id': 658870, 'title': 'EXIT 2 - Directions'},
 {'item_id': 713430, 'title': 'Secrets of Magic 2: Witches and Wizards'},
 {'item_id': 617230, 'title': 'Trapped Within'},
 {'item_id': 681620, 'title': 'Cube Link'},
 {'item_id': 729330, 'title': 'The Girl on the Train'}]